# {County, Muni, Year, Month} aggregation

- Group crashes by {County, Muni, Year, Month}
- Compute sums of {fatal,injury,property} x {victims,crashes}
- Save SQLite (in `www` dir, for use by dev web server)
- Upload to S3 (`s3://nj-crashes/njdot/data/cmym.db`)

In [1]:
from utz import *
from njdot import crashes, vehicles, occupants, pedestrians
from njdot.paths import CMYM_DB, CMYMC_DB
from nj_crashes.utils import sql

In [2]:
%%time
c = crashes.load(cols=['dt', 'year', 'cc', 'mc', 'severity', 'tk', 'ti', 'pk', 'pi', ])
y = c.year.rename('y')
m = c.dt.dt.month.rename('m')
c = sxs(y, m, c[[k for k in c if k not in ['y', 'm', 'year', 'dt']]])
c

Reading /Users/ryan/c/neighbor-ryan/nj-crashes/njdot/data/crashes.parquet


CPU times: user 487 ms, sys: 331 ms, total: 818 ms
Wall time: 686 ms


,y,m,cc,mc,severity,tk,ti,pk,pi
id,,,,,,,,,
0,2001,12,1,1,p,0,0,0,0
1,2001,1,1,1,p,0,0,0,0
2,2001,4,1,1,i,0,4,0,0
3,2001,4,1,1,i,0,1,0,0
4,2001,4,1,1,p,0,0,0,0
...,...,...,...,...,...,...,...,...,...
6077298,2021,5,21,23,p,0,0,0,0
6077299,2021,5,21,23,p,0,0,0,0
6077300,2021,6,21,23,p,0,0,0,0


In [3]:
cmym_cols = [ 'cc', 'mc', 'y', 'm', ]
cmymtc_cols = cmym_cols + [ 'condition', 'type', ]

In [4]:
p = pedestrians.load()
p

Reading /Users/ryan/c/neighbor-ryan/nj-crashes/njdot/data/pedestrians.parquet


,crash_id,pn,condition,city,state,zip,dob,age,sex,alc_test_given,alc_test_type,alc_test_results,charge1,summons1,traffic_controls,cir1,cir2,dir,act,inj_loc,inj_type,med_refused,safety_used,hospital,status1,cyclist,other,charge2,summons2,charge3,summons3,charge4,summons4,status2
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,7,1,3,ABSECON,NJ,08201,07/28/1990,10,M,,,,,,<NA>,<NA>,<NA>,<NA>,41,1,5,<NA>,<NA>,,1,False,False,,,,,,,<NA>
1,48,31,3,ABSECON,NJ,08201,01/16/1967,34,M,,,,,,<NA>,<NA>,<NA>,<NA>,<NA>,1,4,<NA>,<NA>,,<NA>,True,False,,,,,,,<NA>
2,76,1,2,EGG HARBOR TWP.,NJ,08234,11/16/1952,48,F,N,,,,,<NA>,<NA>,<NA>,<NA>,46,12,3,<NA>,<NA>,,1,False,False,,,,,,,<NA>
3,114,1,4,GALLOWAY TWP.,NJ,08205,03/31/1959,42,F,N,,,SUBPOENA,,<NA>,<NA>,<NA>,<NA>,49,7,<NA>,<NA>,<NA>,,1,False,False,,,,,,,<NA>
4,236,1,4,ABSECON,NJ,08201,04/18/1953,48,M,N,,,,,<NA>,<NA>,<NA>,<NA>,43,1,4,<NA>,<NA>,,1,False,False,,,,,,,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178516,6076961,1,4,WASHINGTON,NJ,07882,,74,F,N,,,,,5,25,<NA>,3,43,6,8,2,1,7102,1,False,False,,,,,,,<NA>
178517,6076969,31,3,WASHINGTON,NJ,07882,,17,M,N,,,,,4,25,<NA>,2,3,11,5,1,8,,1,True,False,,,,,,,<NA>
178518,6076973,1,1,PORT MURRAY,NJ,07865,,59,M,N,,,,,4,73,89,0,36,12,3,<NA>,<NA>,7102,2,False,False,,,,,,,<NA>


In [5]:
pm = p.merge(c, left_on='crash_id', right_index=True, how='left', validate='m:1')
pm = pm[(pm.condition >= 1) & (pm.condition <= 5)]
pm.loc[ pm.cyclist, 'type'] = 'b'
pm.loc[~pm.cyclist, 'type'] = 't'
pg = pm.groupby(cmymtc_cols).size().rename('num')
pg

cc  mc  y     m   condition  type
1   1   2001  1   4          t       1
              6   3          t       1
                  4          t       1
              7   4          b       1
              9   3          b       1
                                    ..
21  23  2012  7   4          b       1
        2016  4   1          b       1
        2017  8   4          b       1
        2019  7   2          b       1
        2020  12  1          t       1
Name: num, Length: 80923, dtype: int64

In [6]:
o = occupants.load()
o

Reading /Users/ryan/c/neighbor-ryan/nj-crashes/njdot/data/occupants.parquet


,crash_id,vehicle_id,on,condition,pos,eject,age,sex,inj_loc,inj_type,med_refused,safety_avail,safety_used,airbag,hospital
id,,,,,,,,,,,,,,,
0,0,0,1,<NA>,1,1,38,M,<NA>,<NA>,,4,4,<NA>,
1,0,1,2,<NA>,1,1,63,F,<NA>,<NA>,,4,4,<NA>,
2,1,2,1,<NA>,<NA>,<NA>,<NA>,,<NA>,<NA>,,<NA>,<NA>,<NA>,
3,2,4,1,3,1,1,29,F,6,8,,4,4,<NA>,
4,2,4,2,3,3,1,7,M,8,5,,4,4,<NA>,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13756721,6077301,11450823,1,5,1,1,74,M,<NA>,<NA>,01,11,4,<NA>,
13756722,6077302,11450824,1,5,1,1,23,M,<NA>,<NA>,,11,4,<NA>,
13756723,6077302,11450824,2,5,3,1,23,F,<NA>,<NA>,,11,4,<NA>,


In [7]:
om = o.merge(c, left_on='crash_id', right_index=True, how='left', validate='m:1')
om = om[(om.condition >= 1) & (om.condition <= 5)]
om['type'] = nan
om.loc[om.pos == 1, 'type'] = 'd'
om.loc[om.pos > 1, 'type'] = 'o'
om = om[~om.type.isna()]
og = om.groupby(cmymtc_cols).size().rename('num')
og

/var/folders/dc/gqj_cd8d0d9c1nppd5_yl7bc0000gn/T/ipykernel_95650/3748009522.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'd' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  om.loc[om.pos == 1, 'type'] = 'd'


cc  mc  y     m   condition  type
1   1   2001  1   3          d        3
                  4          d        6
                             o        4
              2   2          d        1
                  4          d        6
                                     ..
21  23  2021  11  5          d       15
                             o        6
              12  3          d        1
                  4          d        1
                  5          d       18
Name: num, Length: 358063, dtype: int64

In [8]:
g = pd.concat([ pg, og ]).sort_index()
g

cc  mc  y     m   condition  type
1   1   2001  1   3          d        3
                  4          d        6
                             o        4
                             t        1
              2   2          d        1
                                     ..
21  23  2021  11  5          d       15
                             o        6
              12  3          d        1
                  4          d        1
                  5          d       18
Name: num, Length: 438986, dtype: int64

In [14]:
occ_severity = om.groupby('crash_id')['condition'].min().rename('occ_severity')
ped_severity = pm.groupby('crash_id')['condition'].min().rename('ped_severity')
sev = sxs(occ_severity, ped_severity).min(axis=1).rename('condition')
cs = sxs(c.drop(columns='severity'), sev)
cs['condition'] = cs.condition.fillna(5)
cs

,y,m,cc,mc,tk,ti,pk,pi,condition
0,2001,12,1,1,0,0,0,0,5
1,2001,1,1,1,0,0,0,0,5
2,2001,4,1,1,0,4,0,0,3
3,2001,4,1,1,0,1,0,0,4
4,2001,4,1,1,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...
6077298,2021,5,21,23,0,0,0,0,5
6077299,2021,5,21,23,0,0,0,0,5
6077300,2021,6,21,23,0,0,0,0,5
6077301,2021,6,21,23,0,0,0,0,5


In [15]:
cxs = cs.groupby(cmym_cols + ['condition']).size().rename('num_crashes')
cxs

cc  mc  y     m   condition
1   1   2001  1   3             2
                  4             8
                  5            10
              2   2             1
                  4             5
                               ..
21  23  2021  11  4             1
                  5            10
              12  3             1
                  4             1
                  5            11
Name: num_crashes, Length: 358449, dtype: int64

In [16]:
w = g.reset_index(level=5).pivot(columns='type', values='num')[['d', 'o', 't', 'b']].rename(columns={
    'd': 'drivers',
    'o': 'passengers',
    't': 'pedestrians',
    'b': 'cyclists',
})
w = sxs(w, cxs).fillna(0).astype(int)
w

drivers  passengers  pedestrians  cyclists  \
cc mc y    m  condition                                               
1  1  2001 1  3                3           0            0         0   
              4                6           4            1         0   
           2  2                1           0            0         0   
              4                6           2            0         0   
           3  3                2           1            0         0   
...                          ...         ...          ...       ...   
21 23 2018 8  5                0           0            0         0   
           9  5                0           0            0         0   
           10 5                0           0            0         0   
           11 5                0           0            0         0   
           12 5                0           0            0         0   

                         num_crashes  
cc mc y    m  condition               
1  1  2001 1  3                    2  
              4                    8  
           2  2                    1  
              4                    5  
           3  3                    2  
...                              ...  
21 23 2018 8  5                    8  
           9  5                   10  
           10 5                   16  
           11 5                   21  
           12 5                    9  

[362087 rows x 5 columns]

In [17]:
hc = w.loc[9].reset_index()
hcf = hc[hc.condition == 1]
hcf

,mc,y,m,condition,drivers,passengers,pedestrians,cyclists,num_crashes
6,1,2001,3,1,2,0,0,0,2
16,1,2001,7,1,2,0,0,0,2
32,1,2001,12,1,1,0,0,0,1
74,1,2003,5,1,0,0,1,0,1
130,1,2005,1,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...
6903,12,2019,5,1,0,0,1,0,1
6908,12,2019,6,1,0,1,0,0,1
6982,12,2021,5,1,0,0,1,0,1
7001,12,2021,10,1,0,0,1,0,1


In [18]:
hcf.sum()

mc               2592
y              794387
m                2609
condition         395
drivers           192
passengers         77
pedestrians       202
cyclists           20
num_crashes       466
dtype: Int64

In [19]:
%%time
sql.write(
    w, 'cmymc', CMYMC_DB,
    idxs=[('cc', 'mc', 'y', 'm', 'condition')],
    rm=True,
    page_size=2**16,
)

Removing /Users/ryan/c/neighbor-ryan/nj-crashes/www/public/njdot/cmymc.db
Writing 362087 rows to /Users/ryan/c/neighbor-ryan/nj-crashes/www/public/njdot/cmymc.db
Wrote DB: 29347840 bytes
After indices: 36167680 bytes


CPU times: user 2.35 s, sys: 261 ms, total: 2.62 s
Wall time: 2.72 s


After setting page_size=65536 and vacuum: 34603008 bytes


In [20]:
v = vehicles.load()
v

Reading /Users/ryan/c/neighbor-ryan/nj-crashes/njdot/data/vehicles.parquet


,crash_id,vn,ins_co,owner_state,make,model,color,vy,state,rm_by,impact_loc,damage_loc,type,use,cargo_type,cir1,cir2,dir,act,ev1,ev2,ev3,ev4,oversize,hit_run,departure,damage,ev
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,1,426,NJ,NISSAN MAXIMA,,BUR,1991,NJ,<NA>,8,7,1,,<NA>,25,<NA>,1,3,26,<NA>,<NA>,<NA>,<NA>,False,1,<NA>,<NA>
1,0,2,989,NJ,LINCOLN TOWNCAR,,BK,1996,NJ,2,12,12,6,,0,4,<NA>,2,3,26,<NA>,<NA>,<NA>,0,False,1,<NA>,<NA>
2,1,1,962,NJ,TOYOTA 4DR,,GRN,1997,NJ,1,11,<NA>,1,,<NA>,25,<NA>,3,10,28,<NA>,<NA>,<NA>,<NA>,False,1,<NA>,<NA>
3,1,2,<NA>,,,,,0,,<NA>,0,0,5,,0,2,<NA>,1,1,26,<NA>,<NA>,<NA>,0,False,0,<NA>,<NA>
4,2,1,85,NJ,CHEVY CORSICA,,PUR,1996,NJ,3,8,15,1,,<NA>,25,<NA>,3,1,26,1,<NA>,<NA>,<NA>,False,6,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11450821,6077299,2,962,NJ,HYUNDAI,SONATA,BK,2012,NJ,2,6,6,1,01,<NA>,25,<NA>,4,3,26,<NA>,<NA>,<NA>,<NA>,False,1,3,26
11450822,6077300,1,134,NJ,SUBARU,FORRESTER,SL,2015,NJ,1,11,11,4,01,<NA>,57,<NA>,3,1,24,<NA>,<NA>,<NA>,2,False,1,3,24
11450823,6077301,1,148,NJ,TOYOTA,COROLLA,BK,2009,NJ,1,11,11,1,01,<NA>,57,<NA>,2,1,24,<NA>,<NA>,<NA>,<NA>,False,1,3,24


In [21]:
vm = v.merge(c, left_on='crash_id', right_index=True, how='left', validate='m:1')
vm['towed'] = vm.departure >= 3
vm['impounded'] = (vm.departure == 4) | (vm.departure == 5)
vm['driven'] = vm.departure == 1
vm['disabled'] = (vm.departure == 3) | (vm.departure == 5) | (vm.damage == 4)
vm['condition'] = vm.damage.fillna(0).astype(int)
vm['left'] = vm.departure == 2
vm.loc[vm.disabled, 'condition'] = 4
vm

,crash_id,vn,ins_co,owner_state,make,model,color,vy,state,rm_by,impact_loc,damage_loc,type,use,cargo_type,cir1,cir2,dir,act,ev1,ev2,ev3,ev4,oversize,hit_run,departure,damage,ev,y,m,cc,mc,severity,tk,ti,pk,pi,towed,impounded,driven,disabled,condition,left
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,1,426,NJ,NISSAN MAXIMA,,BUR,1991,NJ,<NA>,8,7,1,,<NA>,25,<NA>,1,3,26,<NA>,<NA>,<NA>,<NA>,False,1,<NA>,<NA>,2001,12,1,1,p,0,0,0,0,False,False,True,<NA>,0,False
1,0,2,989,NJ,LINCOLN TOWNCAR,,BK,1996,NJ,2,12,12,6,,0,4,<NA>,2,3,26,<NA>,<NA>,<NA>,0,False,1,<NA>,<NA>,2001,12,1,1,p,0,0,0,0,False,False,True,<NA>,0,False
2,1,1,962,NJ,TOYOTA 4DR,,GRN,1997,NJ,1,11,<NA>,1,,<NA>,25,<NA>,3,10,28,<NA>,<NA>,<NA>,<NA>,False,1,<NA>,<NA>,2001,1,1,1,p,0,0,0,0,False,False,True,<NA>,0,False
3,1,2,<NA>,,,,,0,,<NA>,0,0,5,,0,2,<NA>,1,1,26,<NA>,<NA>,<NA>,0,False,0,<NA>,<NA>,2001,1,1,1,p,0,0,0,0,False,False,False,<NA>,0,False
4,2,1,85,NJ,CHEVY CORSICA,,PUR,1996,NJ,3,8,15,1,,<NA>,25,<NA>,3,1,26,1,<NA>,<NA>,<NA>,False,6,<NA>,<NA>,2001,4,1,1,i,0,4,0,0,True,False,False,<NA>,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11450821,6077299,2,962,NJ,HYUNDAI,SONATA,BK,2012,NJ,2,6,6,1,01,<NA>,25,<NA>,4,3,26,<NA>,<NA>,<NA>,<NA>,False,1,3,26,2021,5,21,23,p,0,0,0,0,False,False,True,False,3,False
11450822,6077300,1,134,NJ,SUBARU,FORRESTER,SL,2015,NJ,1,11,11,4,01,<NA>,57,<NA>,3,1,24,<NA>,<NA>,<NA>,2,False,1,3,24,2021,6,21,23,p,0,0,0,0,False,False,True,False,3,False
11450823,6077301,1,148,NJ,TOYOTA,COROLLA,BK,2009,NJ,1,11,11,1,01,<NA>,57,<NA>,2,1,24,<NA>,<NA>,<NA>,<NA>,False,1,3,24,2021,6,21,23,p,0,0,0,0,False,False,True,False,3,False


In [22]:
vh = vm.groupby(cmym_cols)[['hit_run', 'towed', 'disabled']].sum()
vh

hit_run  towed  disabled
cc mc y    m                           
1  1  2001 1         0     14         0
           2         0     10         0
           3         0     22         0
           4         0     27         0
           5         0     10         0
...                ...    ...       ...
21 23 2021 8         0      0         3
           9         0      3         5
           10        0      4         9
           11        0      1         7
           12        0      0         8

[132766 rows x 3 columns]

In [23]:
sql.write(
    vh, 'cmymv', CMYMC_DB,
    idxs=[cmym_cols],
    replace=False,
    page_size=2**16,
)

Writing 132766 rows to /Users/ryan/c/neighbor-ryan/nj-crashes/www/public/njdot/cmymc.db
Wrote DB: 43712512 bytes
After indices: 46006272 bytes
After setting page_size=65536 and vacuum: 45285376 bytes


In [24]:
import boto3
s3 = boto3.client('s3')

In [25]:
s3.upload_file(CMYMC_DB, Bucket='nj-crashes', Key=f'njdot/data/{basename(CMYMC_DB)}')